In [ ]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_huggingface import HuggingFaceEndpoint
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
import os
from huggingface_hub import login


# Initialize components
DATA_PATH = "/content"
DB_FAISS_PATH = "/content"
HUGGINGFACE_REPO_ID = "mistralai/Mistral-7B-Instruct-v0.3"

def initialize_components(user_input=None, conversation_history=None): # Added parameters for user_input and conversation_history
    # Load or create vector store
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    if os.path.exists(os.path.join(DB_FAISS_PATH, "index.faiss")):
        db = FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)
    else:
        # Load and process documents if vector store doesn't exist
        loader = DirectoryLoader(DATA_PATH, glob='*.pdf', loader_cls=PyPDFLoader)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        text_chunks = text_splitter.split_documents(documents)
        db = FAISS.from_documents(text_chunks, embedding_model)
        db.save_local(DB_FAISS_PATH)

    # Initialize LLM
    login(token=HF_TOKEN)
    llm = HuggingFaceEndpoint(
        repo_id=HUGGINGFACE_REPO_ID,
        task="text-generation",
        temperature=0.5,
        model_kwargs={"token": HF_TOKEN, "max_length": 512}
    )

    memory = ConversationBufferMemory(
       memory_key="chat_history",
       return_messages=True,
       # output_key=response['result']  # Remove or adjust this line based on your intended usage
       input_key='question', # Setting 'input_key' to store user's questions
       output_key='answer'
   )
    # Socratic-style prompt template
    custom_prompt = PromptTemplate(
        template="""You are a Socratic AI that helps users learn by asking thoughtful questions.
        Use the provided context to guide your questioning, but never give direct answers.

        Context: {context}

        Current conversation: {chat_history}

        User's question: {question}

        Instructions:
        1. Ask a single, thought-provoking question related to the user's query.
        2. The question should encourage reflection and deeper understanding.
        3. Avoid giving direct answers or hints.
        4. Base your next question on the user's response to the previous one, maintaining a Socratic dialogue.
        Example:
        User: What are factorials?
        AI: Hmm, interesting! Can you think of a way to calculate how many different ways you can arrange 3 objects?

        Remember, always encourage the user to explore ideas and avoid giving direct answers.
        Socratic response:""",
        input_variables=["context", "chat_history", "question"]
    )

    # Create the conversational chain
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=db.as_retriever(search_kwargs={'k': 3}),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": custom_prompt},
        return_source_documents=True,
        verbose=True
    )

    return qa_chain

conversation_history = []
print("Socratic Chatbot: Hello! Let's explore a topic together. What would you like to know?\n")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Socratic Chatbot: Great conversation! See you next time. 👋")
        break

    # Get Socratic-style response
    qa_chain = initialize_components(user_input, conversation_history) # Calling with user_input and conversation_history
    response = qa_chain({"question": user_input}) # Calling the run method with user input to get the result.
    print(f"Socratic Chatbot: {response['answer']}") # Accessing the response from the 'result' key

    # Update conversation history
    conversation_history.append({"role": "user", "content": user_input})
    conversation_history.append({"role": "assistant", "content": response['answer']}) # Accessing response from 'result' key